In [1]:
import time
from parsel import Selector
from selenium import webdriver
# from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service as ChromeService
from bs4 import BeautifulSoup

In [2]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [3]:
data_dir = '../data/'

In [4]:
# configure webdriver
options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
options.add_argument("--disable-extensions")
options.add_argument("--disable-gpu")
options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [5]:
#Initialize data structures
critic_reviews = {'game_id':[], 'game_name': [], 'score':[], 'author':[], 'date':[], 'summary': []}

#Initialize chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, chrome_options=options)

driver.get("https://www.metacritic.com/browse/games/score/metascore/year/pc/filtered?year_selected=2021&view=detailed&sort=desc&page=0")

game_id = 0
to = 7

while True:
    
    # Games listed on page
    home = driver.current_url
    page_games = driver.find_elements_by_class_name('clamp-summary-wrap')
    game_num = 0
    
    while game_num < len(page_games):
        page_games = driver.find_elements_by_class_name('clamp-summary-wrap')
        game_link = page_games[game_num].find_element_by_tag_name('h3')
        game_link.click()
        
        # Game Details
        rev = WebDriverWait(driver, timeout=to).until(lambda d: d.find_element_by_link_text('Critic Reviews'))
        rev.click()
        
        # Fill in game attributes
        game_name =(
            WebDriverWait(driver, timeout=to).until(lambda d: d.find_element_by_tag_name('h1').text)
        )
        
        page = requests.get(driver.current_url)
        soup=BeautifulSoup(driver.page_source, 'html.parser')
        
        reviews = soup.find(class_='body product_reviews')
        
        score_elems = reviews.find_all(class_="review_grade")
        
        for elem in score_elems:
            critic_reviews['game_id'].append(game_id)
            critic_reviews['game_name'].append(game_name)
            critic_reviews['score'].append(int(elem.text))
        
        author_elems = reviews.find_all(class_="source")
        
        for elem in author_elems:
            critic_reviews['author'].append(elem.text)
            
        date_elems = reviews.find_all(class_="date")
        
        for elem in date_elems:
            critic_reviews['date'].append(elem.text)
            
        sum_elems = reviews.find_all(class_="review_body")
        
        for elem in sum_elems:
            critic_reviews['summary'].append(elem.text.lstrip())
            
        game_num+=1
        game_id+=1
        driver.get(home)
        
    # Go to next page
    try:
        next_button = driver.find_element_by_link_text('next')
        next_button.click()
    except:
        break



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\siraj\.wdm\drivers\chromedriver\win32\98.0.4758.80\chromedriver.exe] found in cache
<ipython-input-5-fa349ab23da7>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options, chrome_options=options)


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=98.0.4758.82)


In [7]:
import pandas as pd

reviews = pd.DataFrame(critic_reviews)
reviews.to_csv(data_dir+'reviews.csv')